In [ ]:
input_dataset = "data/csv/cemetery.csv"
insert_batch_size = 10_000

dataverse = "LocationDb"
table_name = "Locations_RTree_CorrelatedPrefix"

dbHost = "http://localhost:19002"
create_db_path = f"CREATE_DATABASE.sql"
benchmark_plan_path = f"commands/benchmark_plan_{dataverse}_insert_{insert_batch_size}.sql"

# None means no limit
debug_max_batches = None

In [2]:
import random

data =[]
with open(input_dataset, "r") as f:
    f.readline()
    for line in f:
        row = line.strip().split(",")
        id = row[0]
        locationX = row[1]
        locationY = row[2]
        description = row[3]
        data.append([id, locationX, locationY, description])

random.shuffle(data)

In [3]:
import random
import requests
import os

def point(x, y):
    return f'point("{x},{y}")'

def rectangle(x1, y1, x2, y2):
    return f'rectangle("{x1},{y1} {x2},{y2}")'

def generate_insert_command(batch):
    query_lines = []
    query_lines.append(f"INSERT INTO table_name ([")
    for row in batch:
        id = row[0]
        location_x = row[1]
        location_y = row[2]
        description = row[3]
        is_last = (row == batch[-1])
        query_lines.append(f'{{"id": {id}, "location": {point(location_x, location_y)}, "description": "{description}"}}{ "" if is_last else "," }')
    query_lines.append("])")
    
    return "".join(query_lines)

In [4]:
inserted_points = []
current_batch = 0
commands = []
while True:
    if debug_max_batches is not None and current_batch >= debug_max_batches:
        break
    batch_to_insert = data[current_batch * insert_batch_size:(current_batch + 1) * insert_batch_size]
    if not batch_to_insert: # no more data to insert
        break
    
    insert_command = generate_insert_command(batch_to_insert)
    inserted_points.extend(map(lambda x: (x[1], x[2]), batch_to_insert))
    
    commands.append(insert_command)
    
    current_batch += 1
    
# write the commands to a file
benchmark_dir = os.path.dirname(benchmark_plan_path)
if not os.path.exists(benchmark_dir):
    os.makedirs(benchmark_dir)
with open(benchmark_plan_path, "w") as f:
    for command in commands:
        f.write(command + "\n")

In [5]:
import logging
import os

logs_dir = "./logs"
if not os.path.exists(logs_dir):
    os.makedirs(logs_dir, exist_ok=True)


In [12]:
with open(create_db_path, "r") as sql_file:
    create_database_script = sql_file.read()
response = requests.post(f"{dbHost}/query/service", data={'statement': create_database_script})
print(response.text)

{
	"requestID": "3647e87f-da22-4281-a861-9cc524fc76d9",
	"plans":{},
	"status": "success",
	"metrics": {
		"elapsedTime": "1.216100542s",
		"executionTime": "1.024574792s",
		"compileTime": "0ns",
		"queueWaitTime": "0ns",
		"resultCount": 0,
		"resultSize": 0,
		"processedObjects": 0,
		"bufferCacheHitRatio": "0.00%",
		"bufferCachePageReadCount": 0
	}
}



In [13]:
from utils import Logger

logger = Logger(f"{dataverse}_{table_name}_{insert_batch_size}", True);
def execute_command(command, client_context_id=None):
    return requests.post(f"{dbHost}/query/service", data={
        'statement': command,
        'dataverse': dataverse,
        'client_context_id': client_context_id,
    })
with open(benchmark_plan_path, "r") as f:
    index = 0
    for line in f:
        index += 1
        command = line.strip()
        if not command:
            continue
        
        command_type = "insert" if command.startswith("INSERT") else "query"
        event = f"command.{command_type}"
        trace_id = str(index).zfill(10)
        response = execute_command(command.replace("table_name", table_name), client_context_id=trace_id)
        json_response = response.json()
        if response.status_code != 200:
            print(f"Error: {response.json()}")
            logger.log({
                "trace_id": json_response.get("clientContextID"),
                "event": event,
                "http.status": response.status_code,
                "status": json_response.get("status"),
                "metrics": json_response.get("metrics"),
                "error": json_response.get("errors"),
            })
        else:
            logger.log({
                "trace_id": json_response.get("clientContextID"),
                "event": event,
                "http.status": response.status_code,
                "status": json_response.get("status"),
                "result_count": len(json_response.get("results", [])) if json_response.get("results") else None,
                "metrics": json_response.get("metrics"),
            })
                    

1745867455 - INFO - {"trace_id": "0000000001", "event": "command.insert", "http.status": 200, "status": "success", "result_count": null, "metrics": {"elapsedTime": "10.976455131s", "executionTime": "10.138412672s", "compileTime": "0ns", "queueWaitTime": "0ns", "resultCount": 0, "resultSize": 0, "processedObjects": 0, "bufferCacheHitRatio": "0.00%", "bufferCachePageReadCount": 0}}
1745867463 - INFO - {"trace_id": "0000000002", "event": "command.insert", "http.status": 200, "status": "success", "result_count": null, "metrics": {"elapsedTime": "7.370232712s", "executionTime": "6.836677045s", "compileTime": "0ns", "queueWaitTime": "0ns", "resultCount": 0, "resultSize": 0, "processedObjects": 0, "bufferCacheHitRatio": "0.00%", "bufferCachePageReadCount": 0}}
1745867471 - INFO - {"trace_id": "0000000003", "event": "command.insert", "http.status": 200, "status": "success", "result_count": null, "metrics": {"elapsedTime": "7.730150296s", "executionTime": "7.492212003s", "compileTime": "0ns", "